## FUNIL ANALÍTICO CASAINTELIGENTE_NOVOAPP ##

In [ ]:
from google.cloud import bigquery
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from delta import DeltaTable
from functools import reduce
from datetime import datetime, timedelta

In [ ]:
%run /eCare/B2B/observability_commands_b2c

In [ ]:
funcionalidade = "CasaInteligente_novoapp"

read_database = 'ecare_silver.b2c_ga4'
read_table_name = 'silver_ga4_novo_App'

arquivo = write_file_name = f'funil_analitico_ga4'

# Variáveis utilizadas para o observability
#Nesta etapa definindo variáveis necessária para compor o processo de observability
#< < < IMPORTANTE MANTER A VARIAVEL COM UMA STRING VAZIA ATRIBUIDA MESMO QUE A VARIÁVEL NÃO SEJA UTILIZADA NO SCRIPT > > >
write_bq_dataset = ''
write_file_name_bq = f'' # INCLUIR ESTA VARIÁVEL PARA COMPLEMENTAR O OBSERVABILITY

write_database = database = 'ecare_gold.b2c_ga4'
write_table_name = table_name = 'funil_analitico_ga4'

tipo_data = str(getArgument("tipo_data"))
custom_data = str(getArgument("custom_data"))

In [ ]:
def definir_data(tipo_data, custom_data):
    if tipo_data == "0": #Define e data padrão D-1
        data_atual = datetime.today()
        data_text = (datetime.strptime(data_atual.strftime('%Y%m%d'), "%Y%m%d") + timedelta(days=-1)).strftime("%Y%m%d")
        tipo_desc = "Workflow padrão"

    else: # Confere a validade no caso de data customizada e depois verifica a validade do tipo da data.
        if len(custom_data) != 8: #confere se o formato da string de entrada tem 8 digitos
            raise Exception('PARAMETRO DE DATA INVALIDO 1')

        try: # Confere se os 8 digitos informados conseguem formar uma data no padrão "AAAAMMDD"
            custom_data = datetime.strptime(custom_data, '%Y%m%d').strftime('%Y%m%d')
        except:
            raise Exception('PARAMETRO DE DATA INVALIDO 2')
        
        # Após a verificação do formato da data confere se o parâmetro "topo_data" está dentro do esperado
        if tipo_data == "1":
            data_text = custom_data
            tipo_desc = "personalizada no Workflow"
        
        elif tipo_data == '2':
            data_text = custom_data
            tipo_desc = "boot de conferência"

        elif tipo_data == '3':
            data_text = custom_data
            tipo_desc = "personalizada via dbutils"
            
        else:
            raise Exception('PARAMETRO DE DATA INVALIDO 3')
    
    print("TIPO DE DATA DO PROCESSO: " + tipo_data + " ("+ tipo_desc +")")
    print("EXECUÇÃO PARA A DATA: " + data_text)
    
    return data_text
    

In [ ]:
#Função para concatenar os DFs
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
    

In [ ]:
def casainteligente_novoapp(data):
	ga_dados = spark.sql(f"""
    SELECT * 
    FROM {read_database}.{read_table_name} 
    WHERE data = date_format(to_date('{data}','yyyyMMdd'), 'yyyy-MM-dd') AND new_app is true """)
    
	ga_dados.createOrReplaceTempView("dados")
    
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',11) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home'
	""")
	
	df_clk_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Card ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',12) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_vw_non_abrir_o_app_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Abrir o App -- ',13) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'vocePrecisaAbrirOApp_smartHome'
	""")
	
	df_clk_alerta_abrir_o_app = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Alerta Abrir o App ", initcap(split(split(eventLabel, "_")[0], "&")[2]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',14) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%&%_smartHome'
	""")
	
	df_vw_erro_generico_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Erro Genérico -- ',15) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName LIKE '%/something-wrong'
	""")
	
	df_vw_non_erro_generico_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Erro Genérico ** ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',16) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName LIKE '%/something-wrong'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_error'
		AND eventLabel LIKE '%&%&algoDeuErradoPorAqui_somethingWrong'
	""")
	
	df_clk_erro_generico = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Erro Genérico ", initcap(split(eventLabel, "_")[0])),' -- ',17) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName LIKE '%/something-wrong'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_somethingWrong'
	""")
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',18) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home'
	""")
	
	df_clk_casa_inteligente_chip = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Chip ", initcap(split(eventLabel, "_")[0])),' -- ',19) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_clk_casa_inteligente_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Card ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',20) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_link = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Link ", initcap(split(eventLabel, "_")[0])),' -- ',22) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_link'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_clk_casa_inteligente_wifi = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Wifi ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',23) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel LIKE 'selectorNetwork&%_smartHome'
	""")
	
	df_vw_non_abrir_app_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Abrir App -- ',24) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'abrirAppSmartWifi_smartHome'
	""")
	
	df_clk_alerta_abrir_app = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Alerta Abrir App ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',25) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%_smartHome'
	""")
	
	df_vw_non_mapa_de_cobertura_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Mapa de Cobertura -- ',26) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'mapaDeCobertura_smartHome'
	""")
	
	df_clk_alerta_mapa_de_cobertura = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Alerta Mapa de Cobertura ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',27) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&mapaDeCobertura&%_smartHome'
	""")
	
	df_vw_reiniciar_roteador_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Reiniciar Roteador -- ',28) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/smart-wifi/begin-restart-router'
	""")
	
	df_vw_non_reiniciar_roteador_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Reiniciar Roteador -- ',29) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/begin-restart-router'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_info'
		AND eventLabel = 'vocePodeTentaReiniciarSeuRoteador_beginRestartRouter'
	""")
	
	df_clk_reiniciar_roteador = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Reiniciar Roteador ", initcap(split(eventLabel, "_")[0])),' -- ',30) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/begin-restart-router'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_beginRestartRouter'
	""")
	
	df_vw_reiniciando_roteador_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Reiniciando Roteador -- ',31) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/smart-wifi/restarting-router'
	""")
	
	df_vw_non_reiniciando_roteador_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Reiniciando Roteador -- ',32) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/restarting-router'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_success'
		AND eventLabel = 'voceComecouAReiniciarORoteador_restartingRouter'
	""")
	
	df_clk_reiniciando_roteador = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Reiniciando Roteador ", initcap(split(eventLabel, "_")[0])),' -- ',33) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/restarting-router'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_restartingRouter'
	""")
	
	df_vw_detalhes_da_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Detalhes da Rede -- ',34) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
	""")
	
	df_vw_non_detalhes_da_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Detalhes da Rede ** ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',35) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_text'
		AND eventLabel LIKE '%&conectados%&bloqueados%_networkDetails'
	""")
	
	df_clk_detalhes_da_rede = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Detalhes da Rede ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',36) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'acoes&%_networkDetails'
	""")
	
	df_clk_detalhes_da_rede_chip = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Detalhes da Rede Chip ", initcap(split(eventLabel, "_")[0])),' -- ',37) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_networkDetails'
	""")
	
	df_clk_detalhes_da_rede_controle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Detalhes da Rede Controle ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',38) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE 'deviceControl&%_networkDetails'
	""")
	
	df_vw_non_editar_nome_e_senha_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Editar Nome e Senha -- ',39) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_modal'
		AND eventLabel = 'editarNomeESenha_networkDetails'
	""")
	
	df_clk_selecionar_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Selecionar Rede -- ',40) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel = 'selectNetwork_networkDetails'
	""")
	
	df_vw_non_desbloquear_dispositivo_wifi_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Desbloquear Dispositivo Wifi -- ',41) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'desbloquearDispositivo_wifi'
	""")
	
	df_clk_desbloquer_dispositivo_wifi = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Desbloquer Dispositivo Wifi ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',42) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '//napp/smart-home/smart-wifi/wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'aleert&desbloquearDispositivo&%_wifi'
	""")
	
	df_vw_non_compartilhar_senha_de_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Compartilhar Senha de Rede -- ',43) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_info'
		AND eventLabel = 'compartilharSenhaDeRede_networkDetails'
	""")
	
	df_clk_compartilhar_senha_de_rede = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Compartilhar Senha de Rede ", initcap(split(eventLabel, "_")[0])),' -- ',44) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/network-details'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_networkDetails'
	""")
	
	df_vw_editar_nome_e_senha_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Editar Nome e Senha -- ',45) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName LIKE '%/edit-name-and-password'
	""")
	
	df_vw_non_editar_nome_e_senha = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Editar Nome e Senha ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',46) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName LIKE '%/edit-name-and-password'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel LIKE 'passwordError&%_editNameAndPassword'
	""")
	
	df_vw_non_alerta_perda_de_conexao_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Alerta Perda de Conexão -- ',47) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName LIKE '%/edit-name-and-password'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_info'
		AND eventLabel = 'quandoAtualizarOsDadosVoceVaiPerderAConexao_editNameAndPassword'
	""")
	
	df_clk_editar_nome_e_senha = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Editar Nome e Senha ", initcap(split(eventLabel, "_")[0])),' -- ',48) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName LIKE '%/edit-name-and-password'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_editNameAndPassword'
	""")
	
	df_vw_dados_atualizados_com_sucesso_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Dados Atualizados com Sucesso -- ',49) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName LIKE '%/network-data-updated'
	""")
	
	df_vw_non_dados_atualizados_com_sucesso_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Dados Atualizados com Sucesso -- ',50) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName LIKE '%/network-data-updated'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_success'
		AND eventLabel = 'voceAtualizouOsDadosDeRede_networkDataUpdated'
	""")
	
	df_clk_dados_atualizados_com_sucesso = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Dados Atualizados com Sucesso ", initcap(split(eventLabel, "_")[0])),' -- ',51) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName LIKE '%/network-data-updated'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_networkDataUpdated'
	""")
	
	df_vw_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Dispositivos -- ',52) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/devices'
	""")
	
	df_vw_non_algo_deu_errado_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Algo Deu Errado Dispositivos -- ',53) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/devices'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'algoDeuErradoPorAqui_devices'
	""")
	
	df_clk_dispositivos_chips = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Dispositivos Chips ", initcap(split(eventLabel, "_")[0])),' -- ',54) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/devices'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_devices'
	""")
	
	df_clk_dispositivos_controle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Dispositivos Controle ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',57) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/devices'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE 'deviceControl&%_devices'
	""")
	
	df_vw_wifi_convidados_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Wifi Convidados -- ',58) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/smart-wifi/network-details/guest-wifi'
	""")
	
	df_vw_non_wifi_convidados_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Wifi Convidados ** ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',59) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_text'
		AND eventLabel LIKE '%&conectados&%&bloqueados&%_guestWifi'
	""")
	
	df_clk_wifi_convidados_toggle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Wifi Convidados Toggle ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',60) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_toggle'
		AND eventLabel LIKE 'statusDaConfiguracao&%_guestWifi'
	""")
	
	df_clk_wifi_convidados = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Wifi Convidados ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',61) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'acoes&%_guestWifi'
	""")
	
	df_clk_wifi_convidados_chip = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Wifi Convidados Chip ", initcap(split(eventLabel, "_")[0])),' -- ',62) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_guestWifi'
	""")
	
	df_clk_wifi_convidados_controle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Wifi Convidados Controle ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',63) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE 'deviceControl&%_guestWifi'
	""")
	
	df_clk_wifi_convidados_controle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Wifi Convidados Controle ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',64) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE 'deviceControl&%_guestWifi'
	""")
	
	df_vw_non_desbloquear_dispositivo_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Desbloquear Dispositivo -- ',65) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_info'
		AND eventLabel = 'desbloquearDispositivo_guestWifi'
	""")
	
	df_clk_desbloquear_dispositivo = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Desbloquear Dispositivo ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',66) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-wifi/guest-wifi'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&desbloquearDispositivo&%_guestWifi'
	""")
	
	df_vw_erro_ao_atualizar_dados_da_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Erro ao Atualizar Dados da Rede -- ',70) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName LIKE '%/something-wrong-updating-data'
	""")
	
	df_vw_non_erro_ao_atualizar_dados_da_rede_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - Non Erro ao Atualizar Dados da Rede ** ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ** ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',71) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName LIKE '%/something-wrong-updating-data'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_error'
		AND eventLabel LIKE '%&%_somethingWrongUpdatingData'
	""")
	
	df_clk_erro_ao_atualizar_dados_da_rede = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Erro ao Atualizar Dados da Rede ", initcap(split(eventLabel, "_")[0])),' -- ',72) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName LIKE '%/something-wrong-updating-data'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%_somethingWrongUpdatingData'
	""")
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',76) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home'
	""")
	
	df_clk_casa_inteligente = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',77) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel LIKE '%&deviceManagement&%_smartHome'
	""")
	
	df_clk_casa_inteligente = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',78) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel LIKE '%&deviceManagement&%_smartHome'
	""")
	
	df_clk_cassa_inteligente_chip = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Cassa Inteligente Chip ", initcap(split(eventLabel, "_")[0])),' -- ',79) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_clk_casa_inteligente_seletor = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Seletor ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',80) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&selectorNetwork&%_smartHome'
	""")
	
	df_clk_casa_inteligente_link = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Link ", initcap(split(eventLabel, "_")[0])),' -- ',81) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_link'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_vw_non_adicionar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Adicionar Dispositivos -- ',82) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'dispositivosInteligentesAdicionarDispositivos_smartHome'
	""")
	
	df_clk_adicionar_dispositivos = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Adicionar Dispositivos ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',83) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&adicionarDispositivos&%_smartHome'
	""")
	
	df_vw_gestao_de_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Gestão de Dispositivos -- ',84) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
	""")
	
	df_clk_gestao_de_dispositivos_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Gestão de Dispositivos Card ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',85) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel = 'favoritos&ativado|desativado_smartDevicesManagement'
	""")
	
	df_clk_gestao_de_dispositivos_card_outros = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Gestão de Dispositivos Card Outros ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',86) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_card'
		AND eventLabel = 'outrosDispositivos&ativado|desativado_smartDevicesManagement'
	""")
	
	df_clk_gestao_de_dispositivos_card_acoes = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Gestão de Dispositivos Card Ações ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',87) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'acoes&%_smartDevicesManagement'
	""")
	
	df_vw_non_editar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Editar Dispositivos -- ',88) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'editarDispositivos_smartDevicesManagement'
	""")
	
	df_clk_editar_dispositivos = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Editar Dispositivos ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',89) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&editarDispositivos&%_smartDevicesManagement'
	""")
	
	df_vw_non_adicionar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Adicionar Dispositivos -- ',90) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'adicionarNovoDispositivo_smartDevicesManagement'
	""")
	
	df_clk_adicionar_dispositivos = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Adicionar Dispositivos ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',91) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/smart-devices/smart-devices-management'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&adicionarNovoDispositivo&%_smartDevicesManagement'
	""")
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',92) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home'
	""")
	
	df_clk_casa_inteligente_rotina = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Rotina ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',93) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&adicionarRotina_smartHome'
	""")
	
	df_clk_casa_inteligente_rotina_link = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Rotina Link ", initcap(split(eventLabel, "_")[0])),' -- ',94) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_link'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_vw_non_nova_rotina_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Nova Rotina -- ',95) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'adicionarNovaRotina_smartHome'
	""")
	
	df_clk_nova_rotina = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Nova Rotina ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',96) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&adicionarRotina&%_smartHome'
	""")
	
	df_clk_rotina_controle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Rotina Controle ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',97) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE '%&acoes&%_smartHome'
	""")
	
	df_clk_rotina_toggle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Rotina Toggle ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2]), " ", initcap(split(split(eventLabel, "_")[0], "&")[3])),' -- ',98) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_toogle'
		AND eventLabel LIKE '%&acoes&%&disable|enable_smartHome'
	""")
	
	df_vw_non_cena_executada_com_sucesso_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Cena Executada com Sucesso -- ',99) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'aCenaFoiExecutadaComSucesso_smartHome'
	""")
	
	df_clk_casa_inteligente_chip = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Chip ", initcap(split(eventLabel, "_")[0])),' -- ',100) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_chips'
		AND eventLabel LIKE '%_smartHome'
	""")
	
	df_vw_non_algo_deu_errado_rotina_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Algo Deu Errado Rotina -- ',101) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'algoDeuErradoPorAqui_smartHome'
	""")
	
	df_vw_cenas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Cenas -- ',102) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/routines/scenes'
	""")
	
	df_vw_non_cenas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Cenas -- ',103) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_text'
		AND eventLabel LIKE '%&dispositivos_scenes'
	""")
	
	df_clk_cenas_acoes = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Cenas Ações ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',104) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'acoes&%_scenes'
	""")
	
	df_vw_non_editar_cena_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Editar Cena -- ',105) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'editarCena_scenes'
	""")
	
	df_clk_editar_cena = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Editar Cena ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',106) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%_scenes'
	""")
	
	df_vw_non_criar_cena_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Criar Cena -- ',107) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'criarNovaCena_scenes'
	""")
	
	df_clk_criar_cena = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Criar Cena ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',108) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/scenes'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%_scenes'
	""")
	
	df_vw_automacoes_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Automações -- ',110) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '++++++++'
	""")
	
	df_vw_non_automacoes_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Automações ** ", initcap(split(eventLabel, "&")[0])),' -- ',111) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_text'
		AND eventLabel LIKE '%&automacoes_automations'
	""")
	
	df_clk_automacoes_acoes = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Automações Ações ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',112) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'acoes&%_automations'
	""")
	
	df_vw_non_editar_automacao_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Editar Automação -- ',113) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'editarAutomacao_automations'
	""")
	
	df_clk_editar_automacao = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Editar Automação ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',114) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%_automations'
	""")
	
	df_vw_non_criar_automacao_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Criar Automação -- ',115) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'criarNovaAutomacao_automations'
	""")
	
	df_clk_criar_automacao = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Criar Automação ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',116) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'alert&%_automations'
	""")
	
	df_vw_non_alerta_app_automations_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Alerta App Automations -- ',117) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/automations'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_info'
		AND eventLabel = 'asFuncoesDessaAutomacaoSoPodemSerEditadasPeloApp_automations'
	""")
	
	df_vw_todas_as_rotinas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Todas as Rotinas -- ',118) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/all-routines/view-all/display-all-routines'
	""")
	
	df_clk_controle_de_cena = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Controle de Cena ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',120) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_icon'
		AND eventLabel LIKE 'sceneControl&%_displayAllRoutines'
	""")
	
	df_vw_non_algo_deu_errado_todas_as_rotinas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Algo Deu Errado Todas as Rotinas -- ',121) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'algoDeuErradoPorAqui_displayAllRoutines'
	""")
	
	df_clk_todas_as_rotinas_toggle = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Todas as Rotinas Toggle ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',122) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_toggle'
		AND eventLabel LIKE 'automations&%_displayAllRoutines'
	""")
	
	df_vw_non_alerta_sem_cenas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Alerta Sem Cenas -- ',123) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'voceAindaNaoTemCenas_displayAllRoutines'
	""")
	
	df_clk_criar_cena = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Criar Cena ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',124) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'voceAindaNaoTemCenas&%_displayAllRoutines'
	""")
	
	df_vw_non_alerta_sem_automacoes_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Alerta Sem Automações -- ',125) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'voceAindaNaoTemAutomacoes_displayAllRoutines'
	""")
	
	df_clk_criar_automacao = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Criar Automação ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',126) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/routines/display-all-routines'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'voceAindaNaoTemAutomacoes&%_displayAllRoutines'
	""")
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',129) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home'
	""")
	
	df_clk_casa_inteligente_info = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Info ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',130) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_banner = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Banner ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',131) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_banner'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_vw_non_casa_inteligente_banner = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("View - NoN Casa Inteligente Banner ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',132) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_banner'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_nao = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Não ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',133) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_dispositivos = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Dispositivos ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',134) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_banner = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Banner ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',136) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_banner'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_clk_casa_inteligente_apps = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Apps ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',137) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE '%&%_smartHome'
	""")
	
	df_vw_non_algo_deu_errado_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Algo Deu Errado Casa Inteligente -- ',138) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home/'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'algoDeuErradoPorAqui_smartHome'
	""")
	
	df_clk_algo_deu_errado_casa_inteligente = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Algo Deu Errado Casa Inteligente ", initcap(split(split(eventLabel, "_")[0], "&")[1])),' -- ',139) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_button'
		AND eventLabel LIKE 'naoFoiPossivelCarregarAlgunsConteudos&%_smartHome'
	""")
	
	df_vw_casa_inteligente_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente -- ',140) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = '/napp/smart-home/'
	""")
	
	df_clk_casa_inteligente_link = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Inteligente Link ", initcap(split(split(eventLabel, "_")[0], "&")[0]), " ", initcap(split(split(eventLabel, "_")[0], "&")[1]), " ", initcap(split(split(eventLabel, "_")[0], "&")[2])),' -- ',142) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'interaction'
		AND screenName = '/napp/smart-home/'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_click_link'
		AND eventLabel LIKE '%&%&%_smartHome'
	""")
	
	df_vw_casa_conectada_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Conectada -- ',143) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-mp-casa-conectada'
	""")
	
	df_clk_casa_conectada_explore_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Conectada Explore Dispositivos -- ',144) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'entrypoint_explore_dispositivos_conforto'
		AND component_copy = 'explore_dispositivos'
		AND component_type = 'box_icon'
		AND screenName = 'novo-app-mp-casa-conectada'
	""")
	
	df_clk_casa_conectada_mosaico = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Conectada Mosaico ", initcap(component_copy)),' -- ',145) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'vertical_mosaic'
		AND module_name = 'carrosel_casa_conforto'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-casa-conectada'
	""")
	
	df_clk_casa_conectada_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Conectada Card ", initcap(component_copy)),' -- ',146) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'small_card'
		AND module_name = 'carrosel_confira_conforto'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-casa-conectada'
	""")
	
	df_vw_casa_protegida_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Protegida -- ',147) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-mp-casa-protegida'
	""")
	
	df_clk_casa_protegida_explore_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Protegida Explore Dispositivos -- ',148) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'entrypoint_explore_dispositivos_seguranca'
		AND component_copy = 'leve_agora_mais_seguranca'
		AND component_type = 'box_icon'
		AND screenName = 'novo-app-mp-casa-protegida'
	""")
	
	df_clk_casa_protegida_mosaico = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Protegida Mosaico ", initcap(component_copy)),' -- ',149) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'vertical_mosaic'
		AND module_name = 'carrosel_casa_seguranca'
		AND component_copy = 'cameras'
		AND screenName = 'novo-app-mp-casa-protegida'
	""")
	
	df_clk_casa_protegida_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Protegida Card ", initcap(component_copy)),' -- ',150) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'small_card'
		AND module_name = 'carrosel_confira_seguranca'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-casa-protegida'
	""")
	
	df_vw_casa_economica_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Economica -- ',151) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-mp-casa-economica'
	""")
	
	df_clk_casa_economica_explore_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Economica Explore Dispositivos -- ',152) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'entrypoint_explore_dispositivos_seguranca'
		AND component_copy = 'tenha_economia_agora'
		AND component_type = 'box_icon'
		AND screenName = 'novo-app-mp-casa-economica'
	""")
	
	df_clk_casa_economica_mosaico = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Economica Mosaico ", initcap(component_copy)),' -- ',153) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'vertical_mosaic'
		AND module_name = 'carrosel_casa_economia'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-casa-economica'
	""")
	
	df_clk_casa_economica_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Economica Card ", initcap(component_copy)),' -- ',154) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'small_card'
		AND module_name = 'carrosel_confira_economia'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-casa-economica'
	""")
	
	df_vw_acessibilidade_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Acessibilidade -- ',155) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-mp-acessibilidade'
	""")
	
	df_clk_acessibilidade_explore_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Acessibilidade Explore Dispositivos -- ',156) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'entrypoint_explore_dispositivos_acessibilidade'
		AND component_copy = 'autonomia_e_cuidado'
		AND component_type = 'box_icon'
		AND screenName = 'novo-app-mp-acessibilidade'
	""")
	
	df_clk_casa_acessibilidade_mosaico = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Acessibilidade Mosaico ", initcap(component_copy)),' -- ',157) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'vertical_mosaic'
		AND module_name = 'carrosel_casa_acessibilidade'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-acessibilidade'
	""")
	
	df_clk_casa_economica_card = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat(concat("Clique - Casa Economica Card ", initcap(component_copy)),' -- ',158) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_type = 'small_card'
		AND module_name = 'carrosel_confira_acessibilidade'
		AND component_copy LIKE '%'
		AND screenName = 'novo-app-mp-acessibilidade'
	""")
	
	df_vw_casa_inteligente_lampadas_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente Lampadas -- ',159) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-casa-inteligente-lampadas'
	""")
	
	df_clk_casa_inteligente_lampadas_explorar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Inteligente Lampadas Explorar Dispositivos -- ',160) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_copy = 'explorar_dispositivos'
		AND component_type = 'primary_button'
		AND screenName = 'novo-app-casa-inteligente-lampadas'
	""")
	
	df_vw_casa_inteligente_assistentes_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente Assistentes -- ',161) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-casa-inteligente-assistentes'
	""")
	
	df_clk_casa_inteligente_assistentes_explorar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Inteligente Assistentes Explorar Dispositivos -- ',162) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_copy = 'explorar_dispositivos'
		AND component_type = 'primary_button'
		AND screenName = 'novo-app-casa-inteligente-assistentes'
	""")
	
	df_vw_casa_inteligente_repetidores_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente Repetidores -- ',163) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-casa-inteligente-repetidores'
	""")
	
	df_clk_casa_inteligente_repetidores_explorar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Inteligente Repetidores Explorar Dispositivos -- ',164) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_copy = 'explorar_dispositivos'
		AND component_type = 'primary_button'
		AND screenName = 'novo-app-casa-inteligente-repetidores'
	""")
	
	df_vw_casa_inteligente_cameras_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente Cameras -- ',165) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-casa-inteligente-camera'
	""")
	
	df_clk_casa_inteligente_cameras_explorar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Inteligente Cameras Explorar Dispositivos -- ',166) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_copy = 'explorar_cameras'
		AND component_type = 'primary_button'
		AND screenName = 'novo-app-casa-inteligente-camera'
	""")
	
	df_vw_casa_inteligente_alimentador_pet_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - Casa Inteligente Alimentador Pet -- ',167) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'screenView'
		AND screenName = 'novo-app-casa-inteligente-alimentador-pet'
	""")
	
	df_clk_casa_inteligente_alimentador_pet_explorar_dispositivos_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Casa Inteligente Alimentador Pet Explorar Dispositivos -- ',168) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND component_copy = 'explorar_dispositivos'
		AND component_type = 'primary_button'
		AND screenName = 'novo-app-casa-inteligente-alimentador-pet'
	""")
	
	df_vw_non_nao_conseguimos_trazer_informacoes_sobre_a_internet_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Não Conseguimos Trazer Informações Sobre a Internet -- ',169) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_alert'
		AND eventLabel = 'naoConseguimosTrazerAsInformacoesSobreSuaInternet_smartHome'
	""")
	
	df_vw_non_erro_api_loading_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('View - NoN Erro API Loading -- ',170) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'noninteraction'
		AND screenName = '/napp/smart-home'
		AND eventCategory = 'b2c_ecare_napp_smartHome'
		AND eventAction = 'screen_view_text'
		AND eventLabel = 'apiLoadingError_smartHome'
	""")
	
	df_clk_entrada_loja_comprar_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Entrada Loja Comprar -- ',$!10) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'services_summary_comprarscreenName:explore/home'
		AND component_copy = 'casa_inteligente'
		AND  component_type = 'small_card'
	""")
	
	df_clk_entrada_starttab_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Entrada StartTab -- ',$!2) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'main_navigation'
		AND screenName = 'start-tab_for-you'
		AND component_copy = 'smart_home'
		AND component_type = 'icon'
	""")
	
	df_clk_entrada_menumais_ = spark.sql("""select
    	data,
    	hora,
    	'CasaInteligente_novoapp' as Funcionalidade,
    	concat('Clique - Entrada MenuMais -- ',$!4) as Evento,
    	event_name as eventCount,
    	concat(Usuario, ga_session_id) as sessions,
    	Usuario as totalUsers,
    	CPF,
    	MSISDN,
    	novum_user_id,
    	Cidade,
    	CRM,
    	Tipo,
    	Ambiente,
    	appVersion,
    	OS,
    	OS_version,
    	CASE
    	WHEN paymentModels like '%-%' THEN 'multi'
    	ELSE 'mono' END as tipoPlano

    	from dados 

		WHERE 
		event_name = 'user_interaction'
		AND module_name = 'menu_mais_horizontal_mosaic_control'
		AND screenName = 'modular_screen/tab_mais'
		AND  component_copy = 'casa_inteligente'
		AND  component_type = 'horizontal_mosaic'
	""")

	# Concatenando todos os DFs
	df = unionAll(		
	df_vw_casa_inteligente_,		
	df_clk_card,		
	df_vw_non_abrir_o_app_,		
	df_clk_alerta_abrir_o_app,		
	df_vw_erro_generico_,		
	df_vw_non_erro_generico_,		
	df_clk_erro_generico,		
	df_vw_casa_inteligente_,		
	df_clk_casa_inteligente_chip,		
	df_clk_casa_inteligente_card,		
	df_clk_casa_inteligente_link,		
	df_clk_casa_inteligente_wifi,		
	df_vw_non_abrir_app_,		
	df_clk_alerta_abrir_app,		
	df_vw_non_mapa_de_cobertura_,		
	df_clk_alerta_mapa_de_cobertura,		
	df_vw_reiniciar_roteador_,		
	df_vw_non_reiniciar_roteador_,		
	df_clk_reiniciar_roteador,		
	df_vw_reiniciando_roteador_,		
	df_vw_non_reiniciando_roteador_,		
	df_clk_reiniciando_roteador,		
	df_vw_detalhes_da_rede_,		
	df_vw_non_detalhes_da_rede_,		
	df_clk_detalhes_da_rede,		
	df_clk_detalhes_da_rede_chip,		
	df_clk_detalhes_da_rede_controle,		
	df_vw_non_editar_nome_e_senha_,		
	df_clk_selecionar_rede_,		
	df_vw_non_desbloquear_dispositivo_wifi_,		
	df_clk_desbloquer_dispositivo_wifi,		
	df_vw_non_compartilhar_senha_de_rede_,		
	df_clk_compartilhar_senha_de_rede,		
	df_vw_editar_nome_e_senha_,		
	df_vw_non_editar_nome_e_senha,		
	df_vw_non_alerta_perda_de_conexao_,		
	df_clk_editar_nome_e_senha,		
	df_vw_dados_atualizados_com_sucesso_,		
	df_vw_non_dados_atualizados_com_sucesso_,		
	df_clk_dados_atualizados_com_sucesso,		
	df_vw_dispositivos_,		
	df_vw_non_algo_deu_errado_dispositivos_,		
	df_clk_dispositivos_chips,		
	df_clk_dispositivos_controle,		
	df_vw_wifi_convidados_,		
	df_vw_non_wifi_convidados_,		
	df_clk_wifi_convidados_toggle,		
	df_clk_wifi_convidados,		
	df_clk_wifi_convidados_chip,		
	df_clk_wifi_convidados_controle,		
	df_clk_wifi_convidados_controle,		
	df_vw_non_desbloquear_dispositivo_,		
	df_clk_desbloquear_dispositivo,		
	df_vw_erro_ao_atualizar_dados_da_rede_,		
	df_vw_non_erro_ao_atualizar_dados_da_rede_,		
	df_clk_erro_ao_atualizar_dados_da_rede,		
	df_vw_casa_inteligente_,		
	df_clk_casa_inteligente,		
	df_clk_casa_inteligente,		
	df_clk_cassa_inteligente_chip,		
	df_clk_casa_inteligente_seletor,		
	df_clk_casa_inteligente_link,		
	df_vw_non_adicionar_dispositivos_,		
	df_clk_adicionar_dispositivos,		
	df_vw_gestao_de_dispositivos_,		
	df_clk_gestao_de_dispositivos_card,		
	df_clk_gestao_de_dispositivos_card_outros,		
	df_clk_gestao_de_dispositivos_card_acoes,		
	df_vw_non_editar_dispositivos_,		
	df_clk_editar_dispositivos,		
	df_vw_non_adicionar_dispositivos_,		
	df_clk_adicionar_dispositivos,		
	df_vw_casa_inteligente_,		
	df_clk_casa_inteligente_rotina,		
	df_clk_casa_inteligente_rotina_link,		
	df_vw_non_nova_rotina_,		
	df_clk_nova_rotina,		
	df_clk_rotina_controle,		
	df_clk_rotina_toggle,		
	df_vw_non_cena_executada_com_sucesso_,		
	df_clk_casa_inteligente_chip,		
	df_vw_non_algo_deu_errado_rotina_,		
	df_vw_cenas_,		
	df_vw_non_cenas_,		
	df_clk_cenas_acoes,		
	df_vw_non_editar_cena_,		
	df_clk_editar_cena,		
	df_vw_non_criar_cena_,		
	df_clk_criar_cena,		
	df_vw_automacoes_,		
	df_vw_non_automacoes_,		
	df_clk_automacoes_acoes,		
	df_vw_non_editar_automacao_,		
	df_clk_editar_automacao,		
	df_vw_non_criar_automacao_,		
	df_clk_criar_automacao,		
	df_vw_non_alerta_app_automations_,		
	df_vw_todas_as_rotinas_,		
	df_clk_controle_de_cena,		
	df_vw_non_algo_deu_errado_todas_as_rotinas_,		
	df_clk_todas_as_rotinas_toggle,		
	df_vw_non_alerta_sem_cenas_,		
	df_clk_criar_cena,		
	df_vw_non_alerta_sem_automacoes_,		
	df_clk_criar_automacao,		
	df_vw_casa_inteligente_,		
	df_clk_casa_inteligente_info,		
	df_clk_casa_inteligente_banner,		
	df_vw_non_casa_inteligente_banner,		
	df_clk_casa_inteligente_nao,		
	df_clk_casa_inteligente_dispositivos,		
	df_clk_casa_inteligente_banner,		
	df_clk_casa_inteligente_apps,		
	df_vw_non_algo_deu_errado_casa_inteligente_,		
	df_clk_algo_deu_errado_casa_inteligente,		
	df_vw_casa_inteligente_,		
	df_clk_casa_inteligente_link,		
	df_vw_casa_conectada_,		
	df_clk_casa_conectada_explore_dispositivos_,		
	df_clk_casa_conectada_mosaico,		
	df_clk_casa_conectada_card,		
	df_vw_casa_protegida_,		
	df_clk_casa_protegida_explore_dispositivos_,		
	df_clk_casa_protegida_mosaico,		
	df_clk_casa_protegida_card,		
	df_vw_casa_economica_,		
	df_clk_casa_economica_explore_dispositivos_,		
	df_clk_casa_economica_mosaico,		
	df_clk_casa_economica_card,		
	df_vw_acessibilidade_,		
	df_clk_acessibilidade_explore_dispositivos_,		
	df_clk_casa_acessibilidade_mosaico,		
	df_clk_casa_economica_card,		
	df_vw_casa_inteligente_lampadas_,		
	df_clk_casa_inteligente_lampadas_explorar_dispositivos_,		
	df_vw_casa_inteligente_assistentes_,		
	df_clk_casa_inteligente_assistentes_explorar_dispositivos_,		
	df_vw_casa_inteligente_repetidores_,		
	df_clk_casa_inteligente_repetidores_explorar_dispositivos_,		
	df_vw_casa_inteligente_cameras_,		
	df_clk_casa_inteligente_cameras_explorar_dispositivos_,		
	df_vw_casa_inteligente_alimentador_pet_,		
	df_clk_casa_inteligente_alimentador_pet_explorar_dispositivos_,		
	df_vw_non_nao_conseguimos_trazer_informacoes_sobre_a_internet_,		
	df_vw_non_erro_api_loading_,		
	df_clk_entrada_loja_comprar_,		
	df_clk_entrada_starttab_,		
	df_clk_entrada_menumais_,
    )  
    
	return df
    

In [ ]:
#teste = casainteligente_novoapp(20240120)
#teste.createOrReplaceTempView("validar")

In [ ]:
#%sql
    
#select distinct Evento, count(distinct sessions) from validar
#group by 1

In [ ]:
def gravar_dados_lake_db(dados, data):
    #Deletando dados duplicados
    spark.sql(f"""
    DELETE 
    FROM {database}.{table_name} 
    WHERE data = date_format(to_date('{data}','yyyyMMdd'), 'yyyy-MM-dd') 
    AND Funcionalidade like '{funcionalidade}'
    """)

    #Gravando dados no DataLake
    dados.write.saveAsTable(
        name=f'{write_database}.{write_table_name}',
        overwrite=False,
        format='delta',
        partitionBy=['data', 'Funcionalidade'],
        mode='append',
        mergeSchema=True)

In [ ]:
# ENCAPSULAR TODAS AS FUNÇÕES QUE COMPÕEM A QUERY PRINCIPAL DENTRO DA FUNÇÃO "chamada_query_principal"
def chamada_query_principal(data): # NÃO ALTERAR O NOME DA FUNÇÃO
    df = casainteligente_novoapp(data)

    return df, 'sucesso'
    

In [ ]:
# FUNÇÃO PARA CHAMAR AS FUNÇÕES DESTE SCRIPT ("definir_data", "chamada_query_principal", "gravar_dados_lake_db", "gravar_dados_bq")
# O retorno d função "definir_data" é a data no formato string (AAAAMMDD), 
# fazer as transformações necessárias dentro das demais funções que irão utilizar este parâmetro
status, desc_erro = rodar_script() # Esta função está escrita dentro do script "observability_commands_b2c"

# FUNÇÃO PARA CHAMAR OS COMANDO OBSERVABILITY (adiciona os dados coletados durante o job e grava na tabela de observability, inclusive erros)
prepara_log(status, desc_erro) #  # Esta função está dentro do script "observability_commands"
    

In [ ]:
spark.catalog.clearCache()